In [1]:
import pandas as pd
import numpy as np #need?
import math #need?
from jellyfish import jaro_winkler
from py_common_subseq import find_common_subsequences
import numbers
import time
from collections import Counter
from fuzzywuzzy import fuzz

In [ ]:
#TO DO
#Pre-processor to normalize things like state acronym, nicknames, abbreviations
#email match gets counted as a considered name match right off the bat, need method for doing this
#name rarity gets considered, if name is very unique then it should be counted as a match regardless of org tokens

In [2]:
import io
import shutil

with io.open('cupola contacts sample.txt', encoding='utf-8', errors='ignore') as source:
    with io.open('cupola contacts sample_utf.txt', mode='w', encoding='utf-8') as target:
        shutil.copyfileobj(source,target)
        
#with io.open('ASAE Tech.csv', encoding='utf-8', errors='ignore') as source:
#    with io.open('ASAE Tech_utf.csv', mode='w', encoding='utf-8') as target:
#        shutil.copyfileobj(source,target)

In [3]:
left_df = pd.read_table('cupola contacts sample_utf.txt',keep_default_na=False)
right_df = pd.read_csv('ASAE Tech_utf.csv',keep_default_na=False)

In [4]:
left_df.head()

,org_person_id,organization_id,status,person_id,org_name,acronym,first_name,middle_name,last_name,position,email,address_id,address1,address2,city,state,postal_code,alt_name,phone
0,245871,79319,2,253411,New York Graduate Admissions Professionals,NULL,Tonya,NULL,Guzman,President and Website,NULL,165507,P.O. Box 14605,NULL,Lenexa,KS,66285,NULL,#N/A
1,377275,54864,2,380239,Alabama State Bar,ASB,Sherry,NULL,Langley,Receptionist,sherry.langley@alabar.org,192053,P.O. Box 671,NULL,Montgomery,AL,36101,NULL,334-269-1515
2,287731,92462,2,294754,Safeguard Properties,,Kellie,NULL,Chambers,"Assistant Vice President, Investor Relations",kellie.chambers@safeguardproperties.com,178242,7887 Safeguard Cir.,NULL,Valley View,OH,44125,NULL,216-520-1334
3,279710,90629,2,286839,National Council on Geocosmic Research,NCGR,Ken,NULL,Irving,Webmaster,ncgrweb@gmail.com,175831,1351 Maryland Ave. NE,Apt. B,Washington,DC,20002-4439,NULL,212-838-6247
4,263389,86595,2,270769,CSSI Inc,CSSI,Cynthia,Anne,Castillo,Chief Executive Officer,NULL,170722,425 Third St. SW,Suite 700,Washington,DC,20024,NULL,202-863-2175


In [5]:
left_df.shape

(4099, 19)

In [12]:
right_df.dtypes

ID                                                                                                                                         int64
SALES_DATE                                                                                                                                object
FIRST_NAME                                                                                                                                object
LAST_NAME                                                                                                                                 object
ATTENDEE_CREDENTIALS                                                                                                                      object
ATTENDEE_TITLE                                                                                                                            object
COMPANY_NAME                                                                                                                      

In [15]:
#I think I need to revisit the approach
#maybe, do a token match on person name only, and seperately do a token match on organization related things
#then join matched keys and only consider matches which exist in BOTH
#use position as a means to score?
#email treated same as phone?  or, maybe ignore altogether since email matches are generally reliable and straightforward?

start_time = time.time()
print "LOADING DATAFRAMES INTO MEMORY..."

left_df = pd.read_table('cupola contacts sample_utf.txt',keep_default_na=False)
right_df = pd.read_csv('ASAE Tech_utf.csv',keep_default_na=False)

#rename dataframe columns for processing:
left_df.rename(columns={'org_person_id':'id',
                        'org_name':'l_org_name',
                        'acronym':'l_acronym',
                        'alt_name':'l_alt_name',
                        'first_name':'l_first_name',
                        'middle_name':'l_middle_name',
                        'last_name':'l_last_name',
                        'position':'l_position',
                        'address1':'l_address1',
                        'address2':'l_address2',
                        'city':'l_city',
                        'state':'l_state',
                        'postal_code':'l_postal_code',
                        'email':'l_email'}, inplace=True)
right_df.rename(columns={'ID':'id',
                         'COMPANY_NAME':'r_org_name',
                         'FIRST_NAME':'r_first_name',
                         'LAST_NAME':'r_last_name',
                         'ATTENDEE_TITLE':'r_position',
                         'ADDRESS1':'r_address1',
                         'ADDRESS2':'r_address2',
                         'CITY':'r_city',
                         'STATE':'r_state',
                         'ZIP_CODE':'r_postal_code',
                         'Email':'r_email'}, inplace=True)

#set parameters
name_token_limiter = 1 #percent of non-single name tokens to tokenize, where common tokens are excluded before rare
org_token_limiter = .995 # percent of non-single org tokens to tokenize, where common tokens are excluded before rare
name_rarity_thresh = .85 
name_token_match_min = 1 # minimum number of matched tokens to be considered a match
org_token_match_min = 1 # minimum number of matched tokens to be considered a match
overall_token_match_min = name_token_match_min + org_token_match_min + 0 #optionally can increase to further limit matches; if +0 this is ignored

#scoring weights
first_name_weight = 1
last_name_weight = 1.25
org_name_weight = .75
zip_weight = 1

print("dataframes loaded --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CREATING DICTIONARY OF ALL UNIQUE TOKENS W INCLUDE FLAG..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# lowercase the name and split on spaces, remove non-alphanumeric chars
def tokenize_name(name):
    if isinstance(name, basestring) is True:
        clean_name = ''.join(c if c.isalnum() else ' ' for c in name)
        return clean_name.lower().split()
    else:
        return name

# same tokenizers as for names, meh, good enough
def tokenize_address(address):
    if isinstance(address, basestring) is True:
        clean_name = ''.join(c if c.isalnum() else ' ' for c in address)
        return clean_name.lower().split()
    else:
        return address

# same tokenizers as for names, meh, good enough
def tokenize_web(web):
    if isinstance(web, basestring) is True:
        clean_name = ''.join(c if c.isalnum() else ' ' for c in web)
        return clean_name.lower().split()
    else:
        return web
    
# for the left dataset

#should we tokenize position?  email?  
#I think email may be better used to exclude from tokenizing process, ie no need to attempt to match if it exists and non generic
left_name_tokenizers = [
    ('l_first_name', tokenize_name),
    ('l_middle_name', tokenize_name),
    ('l_last_name', tokenize_name)
]

left_org_tokenizers = [
    ('l_org_name', tokenize_name),
    ('l_acronym', tokenize_name),
    ('l_alt_name', tokenize_name),
    ('l_city', tokenize_address),
    ('l_state', tokenize_address),
    ('l_postal_code', tokenize_address),
    ('l_position', tokenize_name)
]

# and right
right_name_tokenizers = [
    ('r_first_name', tokenize_name),
    ('r_last_name', tokenize_name)
]

right_org_tokenizers = [
    ('r_org_name', tokenize_name),
    ('r_city', tokenize_address),
    ('r_state', tokenize_address),
    ('r_postal_code', tokenize_address),
    ('r_position', tokenize_name)
]

#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< creating dictionary for name words

name_words = [] #creating list of all words used in name columns for counting to determine rarity

for word in left_df['l_first_name']:
    if isinstance(word, float) is False:
        name_words.append(tokenize_name(str(word)))
        
for word in left_df['l_middle_name']:
    if isinstance(word, float) is False:
        name_words.append(tokenize_name(str(word)))
        
for word in left_df['l_last_name']:
    if isinstance(word, float) is False:
        name_words.append(tokenize_name(str(word)))
        
for word in right_df['r_first_name']:
    if isinstance(word, float) is False:
        name_words.append(tokenize_name(str(word)))

for word in right_df['r_last_name']:
    if isinstance(word, float) is False:
        name_words.append(tokenize_name(str(word)))

flat_name_words = [item for sublist in name_words for item in sublist] #flatten list so it can be counted

#instantiate counter and use to count word frequencies in flat list
cnt = Counter()
for token in flat_name_words:
    cnt[token] += 1
    
name_cnt_dict = dict(cnt) #convert to dictionary

name_tokens_df = pd.DataFrame(name_cnt_dict.items(), columns=['token', 'count'])
name_tokens_df = name_tokens_df.sort_values(by='count')  #sorting by count so that we can take the first x% of tokens by rare frequency

name_token_flag = []
for index, value in enumerate(name_tokens_df['count']):
    if value == 1:
        name_token_flag.append(0)  #for any tokens occuring only once, we exclude
    elif index < int(name_tokens_df.shape[0] * name_token_limiter): #important line, we are cutting the top x% of frequently occuring tokens
        name_token_flag.append(1)
    else:
        name_token_flag.append(0)  #for the most common tokens, we exclude

name_tokens_df['flag'] = name_token_flag

name_tokens_df.drop('count',axis=1,inplace=True)
name_tokens_df['flag'] = name_tokens_df.flag.astype(int) #converting flags to int
name_tokens_dct = name_tokens_df.to_dict('split') #converting tokens_df to dictionary
name_tokens_dct=dict(name_tokens_dct['data']) #honestly can't remember why this works, something to do with conversion to dictionary

#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< creating dictoinary for org words

org_words = [] #creating a list of all words used in org-related columns, for counting to determine rarity

for word in left_df['l_org_name']:
    if isinstance(word, float) is False:
        org_words.append(tokenize_name(str(word)))

for word in left_df['l_alt_name']:
    if isinstance(word, float) is False:
        org_words.append(tokenize_name(str(word)))

for word in left_df['l_city']:
    if isinstance(word, float) is False:
        org_words.append(tokenize_name(str(word)))
        
for word in left_df['l_position']:
    if isinstance(word, float) is False:
        org_words.append(tokenize_name(str(word)))

#for word in left_df['l_email']:
#    if isinstance(word, float) is False:
#        org_words.append(tokenize_name(str(word)))
    
for word in right_df['r_org_name']:
    if isinstance(word, float) is False:
        org_words.append(tokenize_name(str(word)))

for word in right_df['r_city']:
    if isinstance(word, float) is False:
        org_words.append(tokenize_name(str(word)))
        
for word in right_df['r_position']:
    if isinstance(word, float) is False:
        org_words.append(tokenize_name(str(word)))
        
#for word in right_df['r_email']:
#    if isinstance(word, float) is False:
#        org_words.append(tokenize_name(str(word)))

flat_org_words1 = [item for sublist in org_words for item in sublist] #flatten list so it can be counted

#instantiate counter and use to count word frequencies in flat list
cnt = Counter()
for token in flat_org_words1:
    cnt[token] += 1
    
org_cnt_dict = dict(cnt) #convert to dictionary

org_tokens_df = pd.DataFrame(org_cnt_dict.items(), columns=['token', 'count'])
org_tokens_df = org_tokens_df.sort_values(by='count')  #sorting by count so that we can take the first x% of tokens by rare frequency

org_token_flag = []
for index, value in enumerate(org_tokens_df['count']):
    if value == 1:
        org_token_flag.append(0)  #for any tokens occuring only once, we exclude
    elif index < int(org_tokens_df.shape[0] * org_token_limiter): #important line, we are cutting the top x% of frequently occuring tokens
        org_token_flag.append(1)
    else:
        org_token_flag.append(0)  #for the most common tokens, we exclude

org_tokens_df['flag'] = org_token_flag

unique_org_tokens = [] #we treat state and zips differently because we want to include ALl state and zip tokens as these are unique

#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< add chosen unique columns here from each df
for word in left_df['l_state']:
    if isinstance(word, float) is False:
        unique_org_tokens.append(tokenize_name(str(word)))

for word in left_df['l_postal_code']:
    if isinstance(word, float) is False:
        unique_org_tokens.append(tokenize_name(str(word)))

for word in right_df['r_state']:
    if isinstance(word, float) is False:
        unique_org_tokens.append(tokenize_name(str(word)))
        
for word in right_df['r_postal_code']:
    if isinstance(word, float) is False:
        unique_org_tokens.append(tokenize_name(str(word)))

for word in left_df['l_acronym']:
    if isinstance(word, float) is False:
        unique_org_tokens.append(tokenize_name(str(word)))
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        
flat_org_words2 = [item for sublist in unique_org_tokens for item in sublist]
new_unique_tokens = list(set(flat_org_words2) - set(flat_org_words1)) #getting a list of tokens which are NOT included in the first flat_list

org_tokens_df.drop('count',axis=1,inplace=True)
org_tokens_df['flag'] = org_tokens_df.flag.astype(int) #converting flags to int
org_tokens_dct = org_tokens_df.to_dict('split') #converting tokens_df to dictionary
org_tokens_dct=dict(org_tokens_dct['data']) #honestly can't remember why this works, something to do with conversion to dictionary

for token in new_unique_tokens: #assigning flag of 1 for all newly added unique tokens
    org_tokens_dct[token] = 1  #note that we are NOT limiting to tokens occuring more than once, as above.  something that could be enhanced
    
print("token dictionaries created --- %s seconds ---" % (time.time() - start_time))
print ""

#this is the generator function we use to create token/ID pairs
def prepare_name_join_keys(df, tokenizers):
    for source_column, tokenizer in tokenizers:
        if source_column in df.columns:
            for index, record in enumerate(df[source_column]):
                if isinstance(record, numbers.Integral) is False: #control for longs
                    if isinstance(record, float) is False: #control for nans
                        for token in tokenizer(record):
                            if name_tokens_dct[token] == 1: #tokenize only for tokens present in dictionary with value 1
                                yield (token, df.iloc[index]['id'])

def prepare_org_join_keys(df, tokenizers):
    for source_column, tokenizer in tokenizers:
        if source_column in df.columns:
            for index, record in enumerate(df[source_column]):
                if isinstance(record, numbers.Integral) is False: #control for longs
                    if isinstance(record, float) is False: #control for nans
                        for token in tokenizer(record):
                            if org_tokens_dct[token] == 1: #tokenize only for tokens present in dictionary with value 1
                                yield (token, df.iloc[index]['id'])
                            
start_time = time.time()
print "TOKENIZING LEFT NAMES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#NOTE: tokenizing the dataframes is the most computationally expensive part of this script.  

#tokenize left dataframe
left_names_keyed = pd.DataFrame(columns=('token', 'id'))
for item in prepare_name_join_keys(left_df, left_name_tokenizers):
    left_names_keyed.loc[len(left_names_keyed)] = item
    
print("left names tokenized --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "TOKENIZING LEFT ORGS..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#tokenize left dataframe
left_orgs_keyed = pd.DataFrame(columns=('token', 'id'))
for item in prepare_org_join_keys(left_df, left_org_tokenizers):
    left_orgs_keyed.loc[len(left_orgs_keyed)] = item
    
print("left orgs tokenized --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "TOKENIZING RIGHT NAMES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#tokenize right dataframe
right_names_keyed = pd.DataFrame(columns=('token', 'id'))
for item in prepare_name_join_keys(right_df, right_name_tokenizers):
    right_names_keyed.loc[len(right_names_keyed)] = item
    
print("right names tokenized --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "TOKENIZING RIGHT ORGS..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

right_orgs_keyed = pd.DataFrame(columns=('token', 'id'))
for item in prepare_org_join_keys(right_df, right_org_tokenizers):
    right_orgs_keyed.loc[len(right_orgs_keyed)] = item
    
print("right orgs tokenized --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "JOINING LEFT & RIGHT TOKEN KEYS..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#join left and right token keys
name_joined = left_names_keyed.join(right_names_keyed.set_index('token'), on='token', how='inner',lsuffix='_l',rsuffix='_r')
org_joined = left_orgs_keyed.join(right_orgs_keyed.set_index('token'), on='token', how='inner',lsuffix='_l',rsuffix='_r')

print("left & right token keys joined --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "GROUPING BY UNIQUE LEFT & RIGHT IDS & GETTING COUNT OF MATCHED TOKENS..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#aggregate to get a count of unique id_l and id_r pairs based on joined tokens, which is used to assess match strength
aggregations = {
    'id_l': 'count'
}

name_keys_grouped = name_joined.groupby(by=['id_l', 'id_r']).agg(aggregations)
name_keys_grouped.rename(columns={'id_l':'name id_l count'}, inplace=True)
name_matched_records = name_keys_grouped[name_keys_grouped['name id_l count'] >= name_token_match_min]

org_keys_grouped = org_joined.groupby(by=['id_l', 'id_r']).agg(aggregations)
org_keys_grouped.rename(columns={'id_l':'org id_l count'}, inplace=True)
org_matched_records = org_keys_grouped[org_keys_grouped['org id_l count'] >= org_token_match_min]

print("keys grouped & counted --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "JOINING NAME & ORG TOKEN MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

name_org_joined = name_matched_records.join(org_matched_records, on=['id_l', 'id_r'], how='inner')#,lsuffix='_l',rsuffix='_r')

#here we are reducing down to those match candidates which meet the minimum threshold of matched tokens (name + org tokens)
name_org_joined['sum all token matches'] = name_org_joined['name id_l count'] + name_org_joined['org id_l count']
name_org_matched_records = name_org_joined[name_org_joined['sum all token matches'] >= overall_token_match_min]

#adding columns for the id values to extract them from multiindex
left_ids = name_org_matched_records.index.get_level_values(0)
right_ids = name_org_matched_records.index.get_level_values(1)
name_org_matched_records['id_l'] = left_ids
name_org_matched_records['id_r'] = right_ids
#reset index?

print("name & org token matches joined and reduced --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CONCATENATING MATCH IDS WITH ORIGINAL DATA..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#renaming ids to be linked to source.  this happens here and not the beginning so as to play nice with the prepare join keys generator
left_df.rename(columns={'id':'id_l'}, inplace=True)
right_df.rename(columns={'id':'id_r'}, inplace=True)

#creating left/right dataframes which contain only the most relevant details for reviewing the match strengths
left_match_data = left_df[['id_l','l_first_name','l_middle_name','l_last_name','l_org_name','l_position','l_city','l_state','l_postal_code','l_email']].copy()
right_match_data = right_df[['id_r','r_first_name','r_last_name','r_org_name','r_position','r_city','r_state','r_postal_code','r_email']].copy()

#merging matched_records df with original record data for ease of review
l_conc = pd.merge(name_org_matched_records, left_match_data, on='id_l')
full_conc = pd.merge(l_conc, right_match_data, on='id_r')

print("original data concatenated with matches --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "SCORING PERSON NAME SIMULARITY..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on edit distance of org names
def jaro_simularity(left_record, right_record):
    if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
        return jaro_winkler(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '')
def fuzz_partial(left_record, right_record):
    if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
        return fuzz.partial_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)
def fuzz_sort(left_record, right_record):
    if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
        return fuzz.token_sort_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)
def fuzz_set(left_record, right_record):
    if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
        return fuzz.token_set_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)

full_conc['l_first_name'] = full_conc['l_first_name'].astype('str')
#full_conc['l_middle_name'] = full_conc['l_middle_name'].astype('str')
full_conc['l_last_name'] = full_conc['l_last_name'].astype('str')

full_conc['r_first_name'] = full_conc['r_first_name'].astype('str')
#full_conc['r_middle_name'] = full_conc['r_middle_name'].astype('str')
full_conc['r_last_name'] = full_conc['r_last_name'].astype('str')
    
full_conc['first_jaro_score'] = full_conc.apply(lambda x: jaro_simularity(x.l_first_name, x.r_first_name), axis=1)
full_conc['first_fuzz_partial_score'] = full_conc.apply(lambda x: fuzz_partial(x.l_first_name, x.r_first_name), axis=1)
full_conc['first_fuzz_sort_score'] = full_conc.apply(lambda x: fuzz_sort(x.l_first_name, x.r_first_name), axis=1)
full_conc['first_fuzz_set_score'] = full_conc.apply(lambda x: fuzz_set(x.l_first_name, x.r_first_name), axis=1)    
   
full_conc['last_jaro_score'] = full_conc.apply(lambda x: jaro_simularity(x.l_last_name, x.r_last_name), axis=1)
full_conc['last_fuzz_partial_score'] = full_conc.apply(lambda x: fuzz_partial(x.l_last_name, x.r_last_name), axis=1)
full_conc['last_fuzz_sort_score'] = full_conc.apply(lambda x: fuzz_sort(x.l_last_name, x.r_last_name), axis=1)
full_conc['last_fuzz_set_score'] = full_conc.apply(lambda x: fuzz_set(x.l_last_name, x.r_last_name), axis=1)
    
print("person name simularities scored --- %s seconds ---" % (time.time() - start_time))
print ""
    
start_time = time.time()
print "SCORING ORG NAME SIMULARITY..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<    
    
full_conc['l_org_name'] = full_conc['l_org_name'].astype('str')
full_conc['r_org_name'] = full_conc['r_org_name'].astype('str')

full_conc['org_jaro_score'] = full_conc.apply(lambda x: jaro_simularity(x.l_org_name, x.r_org_name), axis=1)
full_conc['org_fuzz_partial_score'] = full_conc.apply(lambda x: fuzz_partial(x.l_org_name, x.r_org_name), axis=1)
full_conc['org_fuzz_sort_score'] = full_conc.apply(lambda x: fuzz_sort(x.l_org_name, x.r_org_name), axis=1)
full_conc['org_fuzz_set_score'] = full_conc.apply(lambda x: fuzz_set(x.l_org_name, x.r_org_name), axis=1)

print("org name simularity scored --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR POSTAL CODE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on matching postal code

def sanitize_postal(postal):
    if isinstance(postal, basestring) is True:
        return ''.join(c for c in (postal or '') if c in '1234567890')
    if isinstance(postal, float) is False:
        return postal

def postal_simularity(postal_a, postal_b):
    sanitized_postal_a = str(sanitize_postal(postal_a))
    sanitized_postal_b = str(sanitize_postal(postal_b))

    # if the number is too short, means it's fubar
    if len(sanitized_postal_a) < 5 or len(sanitized_postal_b) < 5:
        return 0
    if float(max(len(sub) for sub in find_common_subsequences(sanitized_postal_a, sanitized_postal_b))) / 5 >= 1:
        return 1
    else:
        return 0
    
full_conc['zip_match'] = full_conc.apply(lambda x: postal_simularity(x.l_postal_code, x.r_postal_code), axis=1)
    
print("postal codes checked --- %s seconds ---" % (time.time() - start_time))
print ""

#calculate composite match score based on component scores and weights
full_conc['composite_person_name_match_score'] = full_conc.first_jaro_score * first_name_weight \
+ full_conc.first_fuzz_partial_score * first_name_weight \
+ full_conc.first_fuzz_sort_score * first_name_weight \
+ full_conc.first_fuzz_set_score * first_name_weight \
+ full_conc.last_jaro_score * last_name_weight \
+ full_conc.last_fuzz_partial_score * last_name_weight \
+ full_conc.last_fuzz_sort_score * last_name_weight \
+ full_conc.last_fuzz_set_score * last_name_weight 

full_conc['composite_org_match_score'] = full_conc.org_jaro_score * org_name_weight \
+ full_conc.org_fuzz_partial_score * org_name_weight \
+ full_conc.org_fuzz_sort_score * org_name_weight \
+ full_conc.org_fuzz_set_score * org_name_weight \
+ full_conc.zip_match * zip_weight

full_conc['overall_composite_match_score'] = full_conc['composite_person_name_match_score'] + full_conc['composite_org_match_score']

reduced_conc = full_conc[['id_l','id_r','l_first_name','l_middle_name','l_last_name','l_org_name','l_position','r_first_name','r_last_name','r_org_name','r_position','composite_person_name_match_score','composite_org_match_score','overall_composite_match_score']]
reduced_conc.sort_values(by='overall_composite_match_score', ascending=False)

LOADING DATAFRAMES INTO MEMORY...
dataframes loaded --- 0.0490000247955 seconds ---

CREATING DICTIONARY OF ALL UNIQUE TOKENS W INCLUDE FLAG...
token dictionaries created --- 0.206999778748 seconds ---

TOKENIZING LEFT NAMES...
left names tokenized --- 19.5590000153 seconds ---

TOKENIZING LEFT ORGS...
left orgs tokenized --- 91.4609999657 seconds ---

TOKENIZING RIGHT NAMES...
right names tokenized --- 0.583999872208 seconds ---

TOKENIZING RIGHT ORGS...
right orgs tokenized --- 2.95700001717 seconds ---

JOINING LEFT & RIGHT TOKEN KEYS...
left & right token keys joined --- 0.0149998664856 seconds ---

GROUPING BY UNIQUE LEFT & RIGHT IDS & GETTING COUNT OF MATCHED TOKENS...
keys grouped & counted --- 0.058000087738 seconds ---

JOINING NAME & ORG TOKEN MATCHES...


KeyError: 'id_l'

In [8]:
reduced_conc.to_csv('reduced_conc first attempt contact match output test.csv')

In [25]:
name_org_joined.to_csv('name_org_joined.csv')

In [27]:
name_tokens_df.flag.value_counts()

0    3547
1     694
2     387
Name: flag, dtype: int64

In [60]:
name_joined[(name_joined.id_r == 126) & (name_joined.id_l == 417271)]

,token,id_l,id_r
368,melissa,417271,126
3909,thompson,417271,126


In [58]:
org_joined[(org_joined.id_r == 126) & (org_joined.id_l == 417271)]

,token,id_l,id_r
25940,digital,417271,126
25941,strategy,417271,126
1145,neuroscience,417271,126
18027,dc,417271,126


In [31]:
name_org_joined[name_org_joined[[417271,126]]]

KeyError: '[417271    126] not in index'

In [43]:
org_joined[org_joined.id_l == 31681]

,token,id_l,id_r
8888,dc,31681,100
8888,dc,31681,4
8888,dc,31681,5
8888,dc,31681,8
8888,dc,31681,10
8888,dc,31681,13
8888,dc,31681,15
8888,dc,31681,16
8888,dc,31681,21
8888,dc,31681,22


In [15]:
right_df[right_df.id == 11]

,id,SALES_DATE,r_first_name,r_last_name,ATTENDEE_CREDENTIALS,r_position,r_org_name,r_address1,r_address2,r_city,...,WHAT IS YOUR PRIMARY JOB RESPONSIBILITY?,HOW MANY YEARS HAVE YOU ATTENDED THE TECHNOLOGY CONFERENCE & EXPO?,HOW MANY YEARS OF EXPERIENCE DO YOU HAVE IN THE TECHNOLOGY FIELD?,WHAT IS THE MAIN REASON YOU ARE ATTENDING THIS YEAR'S TECHNOLOGY CONFERENCE?,WHAT IS THE STAFF SIZE OF YOUR ORGANIZATION?,WHAT IS YOUR APPROXIMATE TIMELINE FOR PURCHASING YOUR NEXT TECHNOLOGY PRODUCT/SERVICE?,WHAT IS YOUR ORGANIZATION'S ANNUAL TECHNOLOGY BUDGET?,WHAT IS YOUR ROLE IN THE PURCHASING OF PRODUCTS AND SERVICES FOR YOUR EMPLOYER?,OVER THE NEXT YEAR WILL YOUR COMPANY BE LOOKING TO PURCHASE/UPGRADE TECHNOLOGIES IN ANY OF THE FOLLOWING AREAS? (CHECK ALL THAT APPLY),"FOR THIS EVENT, HOW WOULD YOU LIKE TO HEAR FROM OUR SPONSORS AND/OR EXHIBITORS?"
10,11,11/28/2018 11:10 AM,Kevin,McCourt,,"Senior Director, Operations",Association of Clinical Research Professionals,99 Canal Center Plaza #150,,Alexandria,...,Sales,1 - 2,10 -12,See Groundbreaking Technology,75 - 99,1 month - 3 months,"$100,001 - $499,999",Final decision maker,Testing/Research and Survey Srvcs.;Email Marke...,Mail


In [27]:
org_keys_grouped.to_csv('org_keys_grouped.csv')

In [28]:
name_keys_grouped.to_csv('name_keys_grouped.csv')

In [21]:
name_tokens_df['count'].value_counts()

1       3547
2        475
3        170
4        103
5         74
7         33
6         31
8         27
12        20
10        19
9         16
11        15
13         9
17         9
14         8
22         6
18         5
16         5
20         4
27         4
15         4
21         4
33         4
28         3
32         3
19         3
39         3
42         2
25         2
38         2
30         2
23         2
52         2
24         1
36         1
67         1
64         1
29         1
61         1
69         1
51         1
3352       1
26         1
66         1
97         1
Name: count, dtype: int64

In [36]:
mi = pd.MultiIndex.from_arrays((list('abc'), list('def')))
mi.names = ['level_1', 'level_2']

In [39]:
mi

MultiIndex(levels=[[u'a', u'b', u'c'], [u'd', u'e', u'f']],
           labels=[[0, 1, 2], [0, 1, 2]],
           names=[u'level_1', u'level_2'])

In [40]:
mi.get_level_values(0)

Index([u'a', u'b', u'c'], dtype='object', name=u'level_1')

In [46]:
name_org_matched_records.sort_values(by='sum all token matches',ascending=False).head(15)

,,name id_l count,org id_l count,sum all token matches
487591,218,2,7,9
375510,299,2,5,7
417271,126,2,4,6
446026,158,2,3,5
463722,149,1,3,4
41253,22,1,2,3
476343,159,1,2,3
473224,143,1,2,3
469598,62,1,2,3
438638,232,1,2,3


In [52]:
name_org_matched_records.shape

(28, 3)

In [55]:
name_org_matched_records.index.get_level_values(1)

Int64Index([ 14,  22, 116, 152,  77,  24, 195,  79, 205,   9,  14, 151, 169,
            169,  60, 299, 269, 152, 126,  46, 282, 232, 158, 149,  62, 143,
            159, 218],
           dtype='int64')